<a href="https://colab.research.google.com/github/Yyyyyjkim/review_analysis/blob/master/Konlpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install konlpy
!pip install sklearn

In [0]:
import pandas as pd
import pickle
import konlpy
import re
import sklearn as sk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize


In [0]:
with open('/content/drive/My Drive/comment_analysis/data/review.pickle', 'rb') as f:
    review = pickle.load(f)

In [0]:
review.columns = ['ranking','date','desc','asc']
review_tidy = review.applymap(lambda x:re.sub('[\[\]!"#$%&\'()*+,/:;<=>?@\^_`{|}~-✔️🙏🏻👍🤗💕😘]|BEST\n|재구매\n','',x))

In [0]:
review_ranking = list(review_tidy['ranking'])
review_rank_sents = sum([sent.split('\n') for sent in review_ranking],[])

In [0]:
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Mecab
from konlpy.tag import Komoran
from konlpy.tag import Okt

In [0]:
twitter = Okt()
review_rank_noun = [twitter.nouns(sent) for sent in review_rank_sents]

In [0]:
corpus = review_rank_noun

출처: https://excelsior-cjh.tistory.com/93 [EXCELSIOR]

In [0]:
vectorizer = TfidfVectorizer()
tfidf_mat = vectorizer.fit_transform(corpus).toarray()
graph_sentence = np.dot(tfidf_mat, tfidf_mat.T) # 문장 간의 tf-idf

In [0]:
counter = CountVectorizer()
counter_mat = normalize(counter.fit_transform(corpus).toarray().astype(float), axis=0)
vocab = counter.vocabulary_
graph_word = np.dot(counter_mat,counter_mat.T) # 문장 간의 counter 유사도
words = {vocab[word] : word for word in vocab}

In [120]:
graph_word.shape

(1213, 1213)

In [0]:
graph = graph_sentence
# graph = graph_word
matrix_size = graph.shape[0]
for id in range(matrix_size):
  graph[id,id] = 0
  